# 3D Object Reconstruction and Classification


## Colab Environment Setup

For the convenience of the submission and the evaluation, we created a repo for the dependencies and low level functions that are used in the notebook. We claim the [threed-comp](https://github.com/NerdToMars/threed-comp) repo is solely written by ourselves.

Before running this notebook, please run the following command to install the dependencies

In [1]:
# clone the repository and install the dependencies if in Colab
# Colab Setup - Only runs in Google Colab
try:
    import google.colab
    print("Running in Google Colab - setting up environment...")
    
    !uv pip install https://github.com/NerdToMars/threed-comp.git --system
    
    print("✓ Repository cloned and dependencies installed!")
    
except ImportError:
    print("Running locally - using existing environment")


Running locally - using existing environment


In [2]:
import threed_comp as tc
tc.print_authors_info()


Authors:
HUANG chongtian, NTU, Singapore
ZHOU hanzhang, NTU, Singapore
Description: This project is a assignment project for course MA6514


## Dataset Description

[ModelNet10.zip](http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip) is a part of ModelNet40 dataset, containing 4,899 pre-aligned shapes from 10 categories. There are 3,991 (80%) shapes for training and 908 (20%) shapes for testing. The CAD models are in Object File Format (OFF).


We provide following scripts to Download and unzip the dataset (the dataset is few GBs, so it may take a while to download):

In [3]:
# download the dataset to ./datasets folder and unzip it
import urllib.request
import zipfile
from pathlib import Path

# Create datasets directory
datasets_dir = Path("datasets")
datasets_dir.mkdir(exist_ok=True)

# Download the dataset
dataset_url = "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip"
zip_path = datasets_dir / "ModelNet10.zip"

# check if the dataset is already downloaded
if zip_path.exists():
    print(f"Dataset already downloaded to: {zip_path}")
else:
    print("Downloading ModelNet10 dataset...")
    urllib.request.urlretrieve(dataset_url, zip_path)
    print(f"Downloaded to: {zip_path}")

# extract the dataset if not extracted
if not (datasets_dir / "ModelNet10").exists():
    # Extract the dataset
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(datasets_dir)
    print("Extraction complete!")

    # List the contents to verify
    print("\nDataset contents:")
    for item in datasets_dir.iterdir():
        print(f"  {item.name}")

BASE_MODEL_DIR = datasets_dir / "ModelNet10"



print("Using BASE_MODEL_DIR: ", BASE_MODEL_DIR)

Dataset already downloaded to: datasets/ModelNet10.zip
Using BASE_MODEL_DIR:  datasets/ModelNet10


### Metadata of the dataset

We walk through the BASE_MODEL_DIR and aggregate the folder structure of the dataset, the metadata is stored in the metadata_modelnet10.csv file. We commit the metadata_modelnet10.csv file to the [repository](https://github.com/NerdToMars/threed-comp) and included in the packageto avoid walking through the directories next time. Hence we can directly use the utils.get_modelnet10_metadata() function to get the metadata of the dataset.


In [4]:
import threed_comp.utils as utils

# read the metadata of the dataset
metadata = utils.get_modelnet10_metadata()
print(metadata.head())

      object_id    class split                    object_path
0  bathtub_0107  bathtub  test  bathtub/test/bathtub_0107.off
1  bathtub_0108  bathtub  test  bathtub/test/bathtub_0108.off
2  bathtub_0109  bathtub  test  bathtub/test/bathtub_0109.off
3  bathtub_0110  bathtub  test  bathtub/test/bathtub_0110.off
4  bathtub_0111  bathtub  test  bathtub/test/bathtub_0111.off


## Visualize the dataset

### Format of the 3D data in ModelNet10

Object File Format (.off) files are used to represent the geometry of a model by specifying the polygons of the model's surface. The polygons can have any number of vertices.
OFF files are all ASCII files beginning with the keyword OFF. The next line states the number of vertices, the number of faces, and the number of edges. The number of edges can be safely ignored.

The vertices are listed with x, y, z coordinates, written one per line. After the list of vertices, the faces are listed, with one face per line. For each face, the number of vertices is specified, followed by indices into the list of vertices. See the examples below.

Note that earlier versions of the model files had faces with -1 indices into the vertex list. That was due to an error in the conversion program and should be corrected now.
```
OFF numVertices numFaces numEdges
x y z
x y z
... numVertices like above
NVertices v1 v2 v3 ... vN
MVertices v1 v2 v3 ... vM
... numFaces like above

```
Note that vertices are numbered starting at 0 (not starting at 1), and that numEdges will always be zero.


We use [rerun](https://www.rerun.io/) to visualize the dataset. It might ask permission to download the wasm for interactive visualization.

In [5]:
import os
import numpy as np
import trimesh
import rerun as rr


# Reading .off file manually
def read_off(shape_filename):
    with open(os.path.join(BASE_MODEL_DIR, shape_filename), 'r') as file:
    
        if 'OFF' != file.readline().strip():
            raise('Not a valid OFF header')
        n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
        verts = [[float(s) for s in file.readline().strip().split(' ')] for _ in range(n_verts)]
        faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for _ in range(n_faces)]
        file.close()
        
        return np.array(verts), np.array(faces)

def calculate_vertex_normals_trimesh(vertices, faces):
    """
    Calculate vertex normals using trimesh
    
    Args:
        vertices: numpy array of shape (n_vertices, 3)
        faces: numpy array of shape (n_faces, 3) - triangular faces
    
    Returns:
        vertex_normals: numpy array of shape (n_vertices, 3)
    """
    # Create trimesh mesh
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    mesh.fix_normals()
    
    # Get vertex normals
    vertex_normals = mesh.vertex_normals
    
    return vertex_normals


# read a sample of the dataset
verts, faces = read_off(metadata.iloc[10]['object_path'])
print("verts.shape:", verts.shape)
print("faces.shape:", faces.shape)

# Calculate vertex normals using trimesh
vertex_normals = calculate_vertex_normals_trimesh(verts, faces)
print(vertex_normals.shape)

# Visualize the mesh with vertex normals using rerun
rr.init("rerun_notebook")

# rerun log mesh
rr.log("mesh", rr.Mesh3D(vertex_positions=verts, triangle_indices=faces, vertex_normals=vertex_normals, albedo_factor=[0.1, 0.5, 0.5]))

rr.notebook_show()


verts.shape: (6171, 3)
faces.shape: (7415, 3)
(4011, 3)


HTML(value='<div id="371eab1d-fd5f-4abd-affa-d3b6d82969f2"><style onload="eval(atob(\'KGFzeW5jIGZ1bmN0aW9uICgp…

Viewer()